# Downloading and displaying images of stellar clusters

In this notebook, we will download FFIs of several stellar clusters using TESScut and display them with matplotlib. The clusters we will be working with are:
- IC 4725
- M34
- Pleiades
- M7
- NGC 752
- IC 4665
- M24
- IC 2391
- Omega Cen
- 47 Tuc
- M4
- NGC 6752

**Please note** that I edited the documentation of astropy to allow a timeout parameter to be passed to fits.open. If you do not also edit the code in this way, some FITS files may not be downloaded because they are too big. You could solve this either by downloading them via browser or some other method, or by editing the documentation yourself.

If you choose the second option, go throug the error stack until you see errors emerging from your python install's version of _urllib/request.py_. One of the functions that throws the error in this file should have an argument called timeout. Simply edit all the astropy functions higher in the stack so that a timeout argument is passed down from `fits.open` all the way to the urllib function.

## Problems:

- I'm getting timeout errors when I try to download an FFI which is too big.
- Also for big FFIs, a lot of the columns come back as nan. Is this because they're outside the sector?


In [13]:
%matplotlib inline
from astropy.io import fits
from astropy.wcs import WCS
import matplotlib.pyplot as plt
import numpy as np

DEGREES = 0.1
POSITIONS = {"IC 4725":(277.94486, -19.11496), "M34":(40.53083, 42.74615), "Pleiades":(56.86909, 24.10531), 
             "M7":(268.46325, -34.79283), "NGC 752":(29.421, 37.785), "IC 4665": (266.61324, 5.64874),
             "M24": (274.23385, -18.51456), "IC 2391": (130.133, -53.033), "Omega Cen": (201.69121, -47.47686),
             "47 Tuc": (6.022329, -72.081444), "M4": (245.89675, -26.52575), "NGC 6752": (287.71575, -59.98185)}
SECTOR = {"IC 2391":8}
def getSector(name):
    if name in SECTOR:
        return SECTOR[NAME]
    return "All:"

In [21]:
NAME = "NGC 6752"
fits_file = "https://mast.stsci.edu/tesscut/api/v0.1/astrocut?ra={0}&dec={1}&y={2}&x={2}&units=d&sector={3}".format(
    POSITIONS[NAME][0], POSITIONS[NAME][1], DEGREES, getSector(NAME))
print(fits_file)
#fits_file="https://mast.stsci.edu/tesscut/api/v0.1/astrocut?ra=268.46325&dec=-34.79283&y=20&x=20&units=px&sector=All"
#print(fits_file)
#fits.info(fits_file)

https://mast.stsci.edu/tesscut/api/v0.1/astrocut?ra=287.71575&dec=-59.98185&y=0.1&x=0.1&units=d&sector=All:


Open the file, load the image data, and close it.

In [22]:
with fits.open(fits_file, mode = "readonly", timeout=100) as hdulist:
    fits.info(fits_file)
    wcs_info = WCS(hdulist[1].header)
    cal_image = hdulist[1].data['FLUX']
    header = hdulist[1].header
    
#mid_time = 0.5 * (header['TSTOP'] + header['TSTART'])# Use the middle of the exposure
frameIndex = int(0.5 * cal_image.shape[0])

HTTPError: HTTP Error 500: Internal Server Error

Now we can display the ffis:

In [ ]:
plt.figure(figsize = (8,8))

BRIGHTNESS_PERCENTILES = {"M7":[20, 100], "NGC 752":[0, 99], "IC 2391":[0, 99]}# Not sure what the last number does
def get_brightness():
    if NAME in BRIGHTNESS_PERCENTILES:
        return BRIGHTNESS_PERCENTILES[NAME]
    return [4, 98]

#plt.subplot(111, projection = wcs_info)
plt.imshow(cal_image[frameIndex], vmin = np.percentile(cal_image[frameIndex],get_brightness()[0]),
           vmax = np.percentile(cal_image[frameIndex], get_brightness()[1]))
plt.title("TESS Calibrated FFI for " + NAME + ", Frame index %i" % frameIndex);

NGC 752 worked quite well, as did IC 2391. I was able to diagnose the error:

- The read operation timed out (Happens when you try to summon an image that's too big)

but not these errors:

- OSError: corrupt fits file
- HTTPError: HTTP Error 500: Internal Server Error
- OSError: Header missing END card.